In [1]:
import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork

In [13]:
# U = Bernoulli(0.1)
# X = ConditionalBernoulli([U], [0.4, 0.6])
# Y = ConditionalBernoulli([U, X], [[0.3, 0.5], [0.8, 0.2]])
U = Bernoulli()
X = ConditionalBernoulli([U])
Y = ConditionalBernoulli([U, X])
net = BernoulliNetwork({'U': U, 'X': X, 'Y': Y})
print(net)

p(Y | U, X) p(X | U) p(U=0.38)


In [18]:
def ett(net):
	pU1 = net.marginals(X=1)['U'].item()
	pY00, pY01, pY10, pY11 = Y.param.view(-1).tolist()
	return pU1 * (pY01 - pY00) + (1 - pU1) * (pY11 - pY10)
def counferfactual(net):
	abduction = net.marginals(X=1)
	action_net = net.intervene(X=1, U=abduction['U'])
	return action_net.marginals()
def alt_ett(net):
	obs = net.marginals(X=1)
	cf = counferfactual(net)
	return obs['Y'] - cf['Y']

In [19]:
ett(net), alt_ett(net)

(-0.5868473561670111, tensor(0.))

In [26]:
net

p(Y | U, X) p(X | U) p(U=0.38)

In [24]:
net.marginals(X=1)['Y']

tensor(0.2479)

In [22]:
pU = net.marginals(X=1)['U']
pU

tensor(0.6609)

In [23]:
action_net = net.intervene(X=1, U=pU)
action_net.marginals()['Y']

tensor(0.2479)

In [25]:
action_net

p(Y | U, X) p(X=1) p(U=0.66)

In [21]:
obs = net.marginals(X=1)
cf = counferfactual(net)
obs, cf

({'U': tensor(0.6609), 'X': tensor(1.), 'Y': tensor(0.2479)},
 {'U': tensor(0.6609), 'X': tensor(1.), 'Y': tensor(0.2479)})